<a href="https://colab.research.google.com/github/AgaZgo/peft_fine_tuning/blob/main/flan_t5_lora_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install \
    transformers==4.31.0 \
    datasets==2.14.4 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    peft==0.4.0 \
    loralib==0.1.1 \
    accelerate==0.21.0 \
    wandb==0.15.8 \
    ray[tune]==2.6.0 \
    bayesian-optimization -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:0

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch

from datetime import datetime

import numpy as np
import evaluate
import nltk
import torch
import wandb
import os

os.environ['WANDB_LOG_MODEL']='end'
os.environ['WANDB_PROJECT']='sandbox'

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
DATASET_NAME='knkarthick/dialogsum'
MODEL_SIZE='base'

dataset = load_dataset(DATASET_NAME)
model_name = f"google/flan-t5-{MODEL_SIZE}"
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto')  # bf16 only for Ampere GPU
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(examples):
    start_prompt = "Summarize the following dialogue:\n\n"
    end_prompt = "\n\nSummarize:"
    prompts = [start_prompt + dialogue + end_prompt for dialogue in examples['dialogue']]
    examples['input_ids'] = tokenizer(prompts, padding='max_length', truncation=True)['input_ids']
    examples['label'] = tokenizer(examples['summary'], padding='max_length', truncation=True)['input_ids']
    examples['label'] = [
        [-100 if token==tokenizer.pad_token_id else token for token in label] for label in examples['label']
    ]
    return examples

dataset_tokenized = dataset.map(preprocess_data, batched=True)
dataset_tokenized.set_format('torch')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
nltk.download('punkt', quiet=True)

True

In [ ]:
def train_model(config):
    MODEL_SIZE = 'base'
    run_name = f"lora_{datetime.now().strftime('%m%d%H%M')}"

    wandb.init(name=run_name, config=config)


    model_name = f"google/flan-t5-{MODEL_SIZE}"
    base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto')  # bf16 only for Ampere GPU
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def preprocess_data(examples):
        start_prompt = "Summarize the following dialogue:\n\n"
        end_prompt = "\n\nSummarize:"
        prompts = [start_prompt + dialogue + end_prompt for dialogue in examples['dialogue']]
        examples['input_ids'] = tokenizer(prompts, padding='max_length', truncation=True)['input_ids']
        examples['label'] = tokenizer(examples['summary'], padding='max_length', truncation=True)['input_ids']
        examples['label'] = [
            [-100 if token==tokenizer.pad_token_id else token for token in label] for label in examples['label']
        ]
        return examples

    rouge = evaluate.load('rouge')

    def compute_metrics(eval_preds):
        preds, labels = eval_preds

        labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        # sentences seperated with '\n' as required by rougeLsum
        decoded_preds = ["\n".join(nltk.sent_tokenize(preds.strip())) for preds in decoded_preds]
        decoded_labels = ["\n".join(nltk.sent_tokenize(labels.strip())) for labels in decoded_labels]

        results =  rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

        return results

    output_dir = f"./lora_training/{datetime.now().strftime('%y%m%d%H%M')}"

    lora_params = {
        'r': int(config['lora_r']),
        'lora_alpha': int(config['lora_alpha']),
        'target_modules': ['q', 'v'],
        'lora_dropout': 0.05,
        'bias': 'none',
        'task_type': "SEQ_2_SEQ_LM"
    }

    lora_config = LoraConfig(**lora_params)

    lora_model = get_peft_model(
        base_model,
        lora_config
    )

    lora_training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=config['learning_rate'],
        max_steps=10,
        auto_find_batch_size=True,
        logging_steps=5,
        evaluation_strategy='steps',
        # save_steps=50,
        logging_dir=output_dir+"/logs",
        predict_with_generate=True,
        load_best_model_at_end=True,
        disable_tqdm=True,
        report_to='wandb'
        run_name=run_name
    )

    lora_trainer = Seq2SeqTrainer(
        model=lora_model,
        args=lora_training_args,
        train_dataset=dataset_tokenized['train'],
        eval_dataset=dataset_tokenized['validation'],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    lora_trainer.train()
    result = lora_trainer.evaluate()
    wandb.finish()
    return result

In [ ]:
train_model = tune.with_resources(train_model, {"gpu": 1, 'cpu':1})

In [ ]:
search_space = {
    'lora_r': tune.uniform(8,25),
    'lora_alpha': tune.uniform(1,13),
    'learning_rate': 1e-3
}

tune_config = tune.TuneConfig(
    num_samples=4,
    search_alg=BayesOptSearch(metric='eval_loss',mode='min'),
    scheduler=ASHAScheduler(metric='eval_loss', mode='min')
)

tuner = tune.Tuner(
    train_model,
    param_space=search_space,
    tune_config=tune_config
)

In [ ]:
tuner.fit()

2023-08-29 15:05:34,581	INFO tune.py:666 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------------------------+
| Configuration for experiment     train_model_2023-08-29_15-05-31   |
+--------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                   |
| Scheduler                        AsyncHyperBandScheduler           |
| Number of trials                 4                                 |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_2023-08-29_15-05-31
To visualize your results with TensorBoard, run: `tensorboard --logdir /root/ray_results/train_model_2023-08-29_15-05-31`

Trial status: 1 PENDING
Current time: 2023-08-29 15:05:34. Total running time: 0s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs
+-----------------------------------------------------------+
| Trial name             status       lora_r     lora_alpha |
+-----------------------------------------------

(pid=7876) 2023-08-29 15:05:42.576817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Trial train_model_10157ce5 started with configuration:
+-----------------------------------------------+
| Trial train_model_10157ce5 config             |
+-----------------------------------------------+
| learning_rate                           0.001 |
| lora_alpha                            10.3324 |
| lora_r                                12.0382 |
+-----------------------------------------------+



(train_model pid=7876) wandb: Currently logged in as: agazgo. Use `wandb login --relogin` to force relogin
(train_model pid=7876) wandb: wandb version 0.15.9 is available!  To upgrade, please run:
(train_model pid=7876) wandb:  $ pip install wandb --upgrade
(train_model pid=7876) wandb: Tracking run with wandb version 0.15.8
(train_model pid=7876) wandb: Run data is saved locally in /root/ray_results/train_model_2023-08-29_15-05-31/train_model_10157ce5_1_learning_rate=0.0010,lora_alpha=10.3324,lora_r=12.0382_2023-08-29_15-05-34/wandb/run-20230829_150545-qy14zal0
(train_model pid=7876) wandb: Run `wandb offline` to turn off syncing.
(train_model pid=7876) wandb: Syncing run lora_08291505
(train_model pid=7876) wandb: ⭐️ View project at https://wandb.ai/agazgo/sandbox
(train_model pid=7876) wandb: 🚀 View run at https://wandb.ai/agazgo/sandbox/runs/qy14zal0
(train_model pid=7876) Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-ba

...

(train_model pid=7876) {'eval_loss': 1.5143159627914429, 'eval_rouge1': 0.2647286761954122, 'eval_rouge2': 0.0836573576250884, 'eval_rougeL': 0.22919919887289258, 'eval_rougeLsum': 0.24345419741749189, 'eval_runtime': 107.1695, 'eval_samples_per_second': 4.666, 'eval_steps_per_second': 0.588, 'epoch': 0.0}


(train_model pid=7876) wandb: - 11.230 MB of 11.230 MB uploaded (0.000 MB deduped)
(train_model pid=7876) wandb: \ 11.230 MB of 11.230 MB uploaded (0.000 MB deduped)
(train_model pid=7876) wandb: | 11.241 MB of 11.251 MB uploaded (0.000 MB deduped)
(train_model pid=7876) wandb: / 11.241 MB of 11.251 MB uploaded (0.000 MB deduped)


Trial train_model_0d0e1d10 completed after 1 iterations at 2023-08-29 15:28:57. Total running time: 23min 22s
+-----------------------------------------------+
| Trial train_model_0d0e1d10 result             |
+-----------------------------------------------+
| time_this_iter_s                      347.178 |
| time_total_s                          347.178 |
| training_iteration                          1 |
| epoch                                       0 |
| eval_loss                             1.51432 |
| eval_rouge1                           0.26473 |
| eval_rouge2                           0.08366 |
| eval_rougeL                            0.2292 |
| eval_rougeLsum                        0.24345 |
| eval_runtime                          107.169 |
| eval_samples_per_second                 4.666 |
| eval_steps_per_second                   0.588 |
+-----------------------------------------------+

Trial status: 4 TERMINATED
Current time: 2023-08-29 15:28:57. Total running time: 23min 2

(train_model pid=7876) wandb: 
(train_model pid=7876) wandb: Run history:
(train_model pid=7876) wandb:                      eval/loss █▁▁
(train_model pid=7876) wandb:                    eval/rouge1 ▁██
(train_model pid=7876) wandb:                    eval/rouge2 ▁██
(train_model pid=7876) wandb:                    eval/rougeL ▁██
(train_model pid=7876) wandb:                 eval/rougeLsum ▁██
(train_model pid=7876) wandb:                   eval/runtime ▃█▁
(train_model pid=7876) wandb:        eval/samples_per_second ▆▁█
(train_model pid=7876) wandb:          eval/steps_per_second ▆▁█
(train_model pid=7876) wandb:                    train/epoch ▁▁▁▁▁▁
(train_model pid=7876) wandb:              train/global_step ▁▁████
(train_model pid=7876) wandb:            train/learning_rate █▁
(train_model pid=7876) wandb:                     train/loss █▁
(train_model pid=7876) wandb:               train/total_flos ▁
(train_model pid=7876) wandb:               train/train_loss ▁
(train_model pid

ResultGrid<[
  Result(
    metrics={'eval_loss': 1.4855414628982544, 'eval_rouge1': 0.2840209812529511, 'eval_rouge2': 0.09359461581076228, 'eval_rougeL': 0.24282252467012738, 'eval_rougeLsum': 0.26156406370659135, 'eval_runtime': 106.8238, 'eval_samples_per_second': 4.681, 'eval_steps_per_second': 0.59, 'epoch': 0.0, 'done': True, 'trial_id': '10157ce5', 'experiment_tag': '1_learning_rate=0.0010,lora_alpha=10.3324,lora_r=12.0382'},
    path='/root/ray_results/train_model_2023-08-29_15-05-31/train_model_10157ce5_1_learning_rate=0.0010,lora_alpha=10.3324,lora_r=12.0382_2023-08-29_15-05-34',
    checkpoint=None
  ),
  Result(
    metrics={'eval_loss': 1.4840362071990967, 'eval_rouge1': 0.2825821259708068, 'eval_rouge2': 0.09470185046321697, 'eval_rougeL': 0.24406965294983257, 'eval_rougeLsum': 0.26164535710503106, 'eval_runtime': 106.9606, 'eval_samples_per_second': 4.675, 'eval_steps_per_second': 0.589, 'epoch': 0.0, 'done': True, 'trial_id': 'ae1d9d5c', 'experiment_tag': '2_learning_ra